Gun Detection

In [31]:
import numpy as np
import cv2
import imutils
import datetime

gun_cascade = cv2.CascadeClassifier('cascade_gun.xml')
camera = cv2.VideoCapture('data/gun4_2.mp4')
# initialize the first frame in the video stream
firstFrame = None

# loop over the frames of the video

gun_exist = False

while True:
    (grabbed, frame) = camera.read()

    # if the frame could not be grabbed, then we have reached the end of the video
    if not grabbed:
        break

    # resize the frame, convert it to grayscale, and blur it
    frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    
    gun = gun_cascade.detectMultiScale(gray, 1.3, 5, minSize = (100, 100))
    
    if len(gun) > 0:
        gun_exist = True
        
    for (x,y,w,h) in gun:
        frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]    

    # if the first frame is None, initialize it
    if firstFrame is None:
        firstFrame = gray
        continue

    # draw the text and timestamp on the frame
    cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
                    (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

    # show the frame and record if the user presses a key
    cv2.imshow("Security Feed", frame)
    key = cv2.waitKey(1) & 0xFF

if gun_exist:
    print("guns detected")
else:
    print("guns NOT detected")

# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()








guns detected


Knife Detection 

In [28]:
#!/usr/bin/env python3
"""
Combined detector:
 - Knife: Ultralytics YOLO .pt model
 - Gun: OpenCV Haar cascade XML

Usage:
  python detect_gun_knife.py --source 0                    # webcam
  python detect_gun_knife.py --source video.mp4            # local video
  python detect_gun_knife.py --source rtsp://...           # RTSP stream
"""

import argparse
import time
import cv2
import imutils
from ultralytics import YOLO
import numpy as np
import os

# ----------------------------
# Arguments
# ----------------------------
parser = argparse.ArgumentParser()
parser.add_argument("--knife-model", type=str, default="best.pt", help="Path to YOLO .pt for knife")
parser.add_argument("--gun-cascade", type=str, default="cascade_gun.xml", help="Path to gun Haar cascade XML")
# parser.add_argument("--source", type=str, default="data/gun4_2.mp4", help="Video source (0 for webcam or path/rtsp)")
parser.add_argument("--source", type=str, default="data/knife_test_video.mp4", help="Video source (0 for webcam or path/rtsp)")

parser.add_argument("--conf", type=float, default=0.35, help="YOLO confidence threshold")
parser.add_argument("--save", action="store_true", help="Save output video to output.mp4")
parser.add_argument("--device", type=str, default=None, help="Device for YOLO, e.g. 'cpu' or 'cuda:0' (auto if None)")
# args = parser.parse_args() # Comment it out when using in jupyter
args, unknown = parser.parse_known_args()


# ----------------------------
# Load models
# ----------------------------
# YOLO knife model
if not os.path.exists(args.knife_model):
    raise SystemExit(f"Knife model not found: {args.knife_model}")
print("[INFO] Loading YOLO model for knife:", args.knife_model)
yolo_device = args.device if args.device else ("cuda:0" if __import__("torch").cuda.is_available() else "cpu")
model = YOLO(args.knife_model)
# move model to device if required by ultralytics API (model(..., device=...))
print(f"[INFO] YOLO device: {yolo_device}")

# Haar cascade for gun
if not os.path.exists(args.gun_cascade):
    raise SystemExit(f"Gun cascade not found: {args.gun_cascade}")
print("[INFO] Loading Haar cascade for gun:", args.gun_cascade)
gun_cascade = cv2.CascadeClassifier(args.gun_cascade)

# ----------------------------
# Open video source
# ----------------------------
source = args.source
if source.isdigit():
    source = int(source)

cap = cv2.VideoCapture(source)
if not cap.isOpened():
    raise SystemExit(f"[ERROR] Cannot open source: {args.source}")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) or 640)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 480)
fps_in = cap.get(cv2.CAP_PROP_FPS) or 25.0

writer = None
if args.save:
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter("output.mp4", fourcc, fps_in, (width, height))
    print("[INFO] Saving output to output.mp4")

# ----------------------------
# Helper: draw box + label
# ----------------------------
def draw_label(frame, text, box, color=(0, 255, 0), thickness=2):
    x1, y1, x2, y2 = box
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
    (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
    cv2.rectangle(frame, (x1, y1 - th - 6), (x1 + tw + 6, y1), color, -1)
    cv2.putText(frame, text, (x1 + 3, y1 - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

# ----------------------------
# Main loop
# ----------------------------
print("[INFO] Starting detection. Press 'q' to quit.")
prev_time = time.time()
while True:
    grabbed, frame = cap.read()
    if not grabbed:
        break

    # optional resize for speed (keep aspect)
    frame = imutils.resize(frame, width=800)

    # ---- Haar gun detection (works on grayscale) ----
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # tune scaleFactor and minNeighbors to reduce false positives
    guns = gun_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    # draw gun boxes
    for (x, y, w, h) in guns:
        x2, y2 = x + w, y + h
        draw_label(frame, "GUN", (x, y, x2, y2), color=(0, 0, 255))

    # ---- YOLO knife detection ----
    # Ultralytics accepts numpy array frames directly:
    try:
        # predict returns a Results object (list-like). Pass device and conf.
        results = model.predict(source=[frame], conf=args.conf, device=yolo_device, verbose=False)
        # results is a list; take first
        if results and len(results) > 0:
            r = results[0]
            # r.boxes: xyxy, conf, cls
            if hasattr(r, "boxes") and r.boxes is not None and len(r.boxes) > 0:
                boxes = r.boxes.xyxy.cpu().numpy() if hasattr(r.boxes.xyxy, "cpu") else np.array(r.boxes.xyxy)
                confs = r.boxes.conf.cpu().numpy() if hasattr(r.boxes.conf, "cpu") else np.array(r.boxes.conf)
                clss = r.boxes.cls.cpu().numpy() if hasattr(r.boxes.cls, "cpu") else np.array(r.boxes.cls)
                for box, conf, cls in zip(boxes, confs, clss):
                    x1, y1, x2, y2 = map(int, box)
                    label = r.names[int(cls)] if hasattr(r, "names") else ("knife" if int(cls)==0 else str(int(cls)))
                    text = f"{label} {conf:.2f}"
                    draw_label(frame, text, (x1, y1, x2, y2), color=(0, 255, 0))
    except Exception as e:
        # if YOLO fails for a frame, print once and continue
        print("[WARN] YOLO exception:", e)

    # timestamp & FPS
    now = time.time()
    fps = 1.0 / (now - prev_time) if (now - prev_time) > 0 else 0.0
    prev_time = now
    ts = time.strftime("%Y-%m-%d %H:%M:%S")
    cv2.putText(frame, f"{ts}  FPS:{fps:.1f}", (10, frame.shape[0] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 2)

    # show & optionally write
    cv2.imshow("Gun+Knife Detection", frame)
    if writer:
        writer.write(frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
if writer:
    writer.release()
cv2.destroyAllWindows()


[INFO] Loading YOLO model for knife: best.pt
[INFO] YOLO device: cpu
[INFO] Loading Haar cascade for gun: cascade_gun.xml
[INFO] Starting detection. Press 'q' to quit.


In [26]:
#!/usr/bin/env python3
"""
Debounced combined detector:
 - Knife: Ultralytics YOLO .pt
 - Gun: OpenCV Haar cascade (debounced + filters)

Usage:
  python detect_gun_knife_debounced.py --knife-model best.pt --gun-cascade cascade.xml --source data/video.mp4 --save
  python detect_gun_knife_debounced.py --source 0
"""

import argparse
import time
import os
import cv2
import imutils
import numpy as np
from ultralytics import YOLO

# ----------------------------
# Arguments
# ----------------------------
parser = argparse.ArgumentParser()
parser.add_argument("--knife-model", type=str, default="best.pt", help="Path to YOLO .pt for knife")
parser.add_argument("--gun-cascade", type=str, default="cascade_gun.xml", help="Path to gun Haar cascade XML")
parser.add_argument("--source", type=str, default="data/gun4_2.mp4", help="Video source (0 for webcam or path/rtsp)")
parser.add_argument("--conf", type=float, default=0.35, help="YOLO confidence threshold")
parser.add_argument("--save", action="store_true", help="Save output video to output.mp4")
parser.add_argument("--device", type=str, default=None, help="Device for YOLO, e.g. 'cpu' or 'cuda:0'")
args, _ = parser.parse_known_args()

# ----------------------------
# Configurable tuning params
# ----------------------------
# Haar cascade params (tune to reduce false positives)
HAAR_SCALE = 1.25
HAAR_NEIGHBORS = 7
HAAR_MIN_SIZE = (40, 40)

# Filtering & tracking params
MIN_AREA = 900            # min area to accept a detection (px^2)
ASPECT_RANGE = (0.25, 2.5)  # acceptable width/height ratio
MAX_DIST = 60             # px distance to match detections between frames
MIN_HITS = 3              # require at least this many hits before showing a gun box
MAX_AGE = 0.6             # seconds to keep a track without updates

# ----------------------------
# Utility: drawing function
# ----------------------------
def draw_label(frame, text, box, color=(0, 255, 0), thickness=2):
    x1, y1, x2, y2 = box
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
    (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
    cv2.rectangle(frame, (x1, y1 - th - 6), (x1 + tw + 6, y1), color, -1)
    cv2.putText(frame, text, (x1 + 3, y1 - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)

# ----------------------------
# Load models
# ----------------------------
if not os.path.exists(args.knife_model):
    raise SystemExit(f"Knife model not found: {args.knife_model}")
print("[INFO] Loading YOLO model for knife:", args.knife_model)
model = YOLO(args.knife_model)
yolo_device = args.device if args.device else ("cuda:0" if __import__("torch").cuda.is_available() else "cpu")
print(f"[INFO] YOLO device: {yolo_device}")

if not os.path.exists(args.gun_cascade):
    raise SystemExit(f"Gun cascade not found: {args.gun_cascade}")
print("[INFO] Loading Haar cascade for gun:", args.gun_cascade)
gun_cascade = cv2.CascadeClassifier(args.gun_cascade)

# ----------------------------
# Open video source
# ----------------------------
src = int(args.source) if args.source.isdigit() else args.source
cap = cv2.VideoCapture(src)
if not cap.isOpened():
    raise SystemExit(f"[ERROR] Cannot open source: {args.source}")

W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) or 640)
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 480)
FPS = cap.get(cv2.CAP_PROP_FPS) or 25.0

writer = None
if args.save:
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter("output.mp4", fourcc, FPS, (W, H))
    print("[INFO] Saving output to output.mp4")

# ----------------------------
# Gun tracks storage (persist across frames)
# ----------------------------
gun_tracks = []  # each track: {'cx','cy','x1','y1','x2','y2','hits','last_seen'}

# ----------------------------
# Main loop
# ----------------------------
print("[INFO] Starting detection. Press 'q' to quit.")
prev_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = imutils.resize(frame, width=800)
    cur_time = time.time()

    # ---------- Haar: raw detections ----------
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    raw = gun_cascade.detectMultiScale(gray, scaleFactor=HAAR_SCALE,
                                       minNeighbors=HAAR_NEIGHBORS, minSize=HAAR_MIN_SIZE)

    # ---------- Filter & associate to tracks ----------
    new_tracks = []
    # process raw boxes
    for (x, y, w, h) in raw:
        area = w * h
        if area < MIN_AREA:
            continue
        aspect = w / float(h) if h > 0 else 1.0
        if not (ASPECT_RANGE[0] <= aspect <= ASPECT_RANGE[1]):
            continue
        cx, cy = x + w // 2, y + h // 2

        matched = False
        for t in gun_tracks:
            dx = cx - t['cx']; dy = cy - t['cy']
            dist = (dx*dx + dy*dy) ** 0.5
            if dist <= MAX_DIST:
                # update existing track
                t['cx'], t['cy'] = cx, cy
                t['x1'], t['y1'], t['x2'], t['y2'] = x, y, x + w, y + h
                t['hits'] += 1
                t['last_seen'] = cur_time
                new_tracks.append(t)
                matched = True
                break
        if not matched:
            new_t = {'cx': cx, 'cy': cy, 'x1': x, 'y1': y, 'x2': x + w, 'y2': y + h,
                     'hits': 1, 'last_seen': cur_time}
            new_tracks.append(new_t)

    # carry over old tracks not matched but recently seen
    for t in gun_tracks:
        if (cur_time - t['last_seen']) <= MAX_AGE:
            new_tracks.append(t)

    # dedupe/merge nearby tracks
    merged = []
    for t in new_tracks:
        found = False
        for m in merged:
            dx = t['cx'] - m['cx']; dy = t['cy'] - m['cy']
            if (dx*dx + dy*dy) ** 0.5 <= (MAX_DIST / 2):
                # merge: average center, expand bbox, max hits, update last_seen
                m['cx'] = int((m['cx'] + t['cx']) / 2)
                m['cy'] = int((m['cy'] + t['cy']) / 2)
                m['x1'] = min(m['x1'], t['x1']); m['y1'] = min(m['y1'], t['y1'])
                m['x2'] = max(m['x2'], t['x2']); m['y2'] = max(m['y2'], t['y2'])
                m['hits'] = max(m['hits'], t['hits'])
                m['last_seen'] = max(m['last_seen'], t['last_seen'])
                found = True
                break
        if not found:
            merged.append(t)

    # keep only recent tracks
    gun_tracks = [t for t in merged if (cur_time - t['last_seen']) <= MAX_AGE]

    # ---------- Draw persistent gun boxes (only after MIN_HITS) ----------
    for t in gun_tracks:
        if t['hits'] >= MIN_HITS:
            draw_label(frame, "GUN", (t['x1'], t['y1'], t['x2'], t['y2']), color=(0, 0, 255))

    # ---------- YOLO knife detection (single-frame streaming) ----------
    try:
        # model(..., stream=True) yields one Results object for this input frame
        results = model(frame, conf=args.conf, device=yolo_device, stream=True)
        for r in results:
            # r.boxes may be empty
            if hasattr(r, "boxes") and r.boxes is not None and len(r.boxes) > 0:
                # convert tensors to numpy if needed
                boxes = r.boxes.xyxy.cpu().numpy() if hasattr(r.boxes.xyxy, "cpu") else np.array(r.boxes.xyxy)
                confs = r.boxes.conf.cpu().numpy() if hasattr(r.boxes.conf, "cpu") else np.array(r.boxes.conf)
                clss = r.boxes.cls.cpu().numpy() if hasattr(r.boxes.cls, "cpu") else np.array(r.boxes.cls)
                for box, conf, cls in zip(boxes, confs, clss):
                    x1, y1, x2, y2 = map(int, box)
                    label = r.names[int(cls)] if hasattr(r, "names") else f"class{int(cls)}"
                    draw_label(frame, f"{label} {conf:.2f}", (x1, y1, x2, y2), color=(0, 255, 0))
            break
    except Exception as e:
        # don't spam — print once per failure
        print("[WARN] YOLO exception:", e)

    # ---------- FPS & timestamp ----------
    now = time.time()
    fps_val = 1.0 / (now - prev_time) if (now - prev_time) > 0 else 0.0
    prev_time = now
    ts = time.strftime("%Y-%m-%d %H:%M:%S")
    cv2.putText(frame, f"{ts}  FPS:{fps_val:.1f}", (10, frame.shape[0] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 2)

    # ---------- display & save ----------
    cv2.imshow("Gun+Knife (debounced)", frame)
    if writer:
        writer.write(frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# cleanup
cap.release()
if writer:
    writer.release()
cv2.destroyAllWindows()
print("[INFO] Done.")


[INFO] Loading YOLO model for knife: best.pt
[INFO] YOLO device: cpu
[INFO] Loading Haar cascade for gun: cascade_gun.xml
[INFO] Starting detection. Press 'q' to quit.

0: 384x640 (no detections), 102.8ms

0: 384x640 (no detections), 115.3ms

0: 384x640 (no detections), 105.7ms

0: 384x640 (no detections), 88.3ms

0: 384x640 (no detections), 105.6ms

0: 384x640 (no detections), 141.1ms

0: 384x640 1 knife, 94.4ms

0: 384x640 (no detections), 86.0ms

0: 384x640 (no detections), 106.8ms

0: 384x640 (no detections), 123.3ms

0: 384x640 (no detections), 108.2ms

0: 384x640 (no detections), 124.2ms

0: 384x640 (no detections), 101.6ms

0: 384x640 (no detections), 101.7ms

0: 384x640 1 guns, 97.7ms

0: 384x640 2 gunss, 80.9ms

0: 384x640 1 guns, 85.6ms

0: 384x640 1 guns, 167.6ms

0: 384x640 1 guns, 101.6ms

0: 384x640 1 guns, 94.0ms

0: 384x640 1 guns, 90.3ms

0: 384x640 2 gunss, 81.5ms

0: 384x640 2 gunss, 104.7ms

0: 384x640 1 guns, 87.6ms

0: 384x640 3 gunss, 70.4ms

0: 384x640 2 gunss, 